# BIBLIOTECAS USADAS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings
import glob
import os
import collections as cll
%matplotlib inline

warnings.filterwarnings('ignore')

# TRATAMENTO DE DADOS

In [ ]:
%%time
#Leitura de Dados
dados = []

for i in range(6984):
    dados.append(pd.read_csv('dados/positions tratados/sem falha/' + str(i) + '.csv'))
    
for i in range(1972):
    dados.append(pd.read_csv('dados/positions tratados/falha 1/' + str(i) + '.csv'))

for i in range(1497):
    dados.append(pd.read_csv('dados/positions tratados/falha 2/' + str(i) + '.csv'))

for i in range(681):
    dados.append(pd.read_csv('dados/positions tratados/falha 4/' + str(i) + '.csv'))
        
for i in range(12):
    dados.append(pd.read_csv('dados/positions tratados/falha 5/' + str(i) + '.csv'))

for i in range(7):
    dados.append(pd.read_csv('dados/positions tratados/falha 6/' + str(i) + '.csv'))

for i in range(7):
    dados.append(pd.read_csv('dados/positions tratados/falha 7/' + str(i) + '.csv'))
    
for i in range(36):
    dados.append(pd.read_csv('dados/positions tratados/falha 8/' + str(i) + '.csv'))
        
for i in range(196):
    dados.append(pd.read_csv('dados/positions tratados/falha 9/' + str(i) + '.csv'))
        
for i in range(1400):
    dados.append(pd.read_csv('dados/positions tratados/falha 10/' + str(i) + '.csv'))
        
for i in range(3):
    dados.append(pd.read_csv('dados/positions tratados/falha 11/' + str(i) + '.csv'))         
    
for i in range(51):
    dados.append(pd.read_csv('dados/positions tratados/falha 12/' + str(i) + '.csv')) 
        
for i in range(1500):
    dados.append(pd.read_csv('dados/positions tratados/falha 13/' + str(i) + '.csv')) 

In [3]:
#Cria DataFrame
df = pd.DataFrame()
n = len(dados)

12586


In [4]:
%%time
#Limpando colunas desnecessárias
for i in range(n):
    dados[i].drop(dados[i].columns.difference(['latitude',
                               'longitude',
                               'velocidade',
                               'ignicao',
                               'data_servidor',
                               'odometro',
                               'bateria',
                               'idx_memoria',
                               'sinal_gps',
                               'serial']), axis = 1, inplace=True)

CPU times: user 12.4 s, sys: 37.3 ms, total: 12.5 s
Wall time: 12.6 s


In [ ]:
#Coloca data no DataFrame
data = []
for i in range(n):
    data.append(dados[i]['data_servidor'][0].split(' ')[0])
df['data'] = data

In [ ]:
#Coloca serial do DataFrame
serial = []
for i in range(n):
    serial.append(dados[i]['serial'][0])
df['serial'] = serial

In [5]:
#Coloca a quantidade de pontos no DataFrame
quantidade_pontos = []
for i in range(n):
    quantidade_pontos.append(dados[i].shape[0])
df['quantidade_pontos'] = quantidade_pontos

In [ ]:
%%time
quantidade_pulos_gps = []
for i in range(n):
    cont = 0
    for j in range(df['quantidade_pontos'][i] - 1):
        pul_lon = False
        pul_lat = False
        if abs(float(dados[i]['longitude'][j]) - float(dados[i]['longitude'][j+1])) > 0.6:
            pul_lon = True
        if abs(float(dados[i]['latitude'][j]) - float(dados[i]['latitude'][j+1])) > 0.6:
            pul_lat = True
        if pul_lat or pul_lon:
            cont +=1
    quantidade_pulos_gps.append(cont)
df['quantidade_pulos_gps'] = quantidade_pulos_gps

In [ ]:
#Coloca variacao da longitude no DataFrame
variacao_lon = []
tmp = False
for i in range(n):
    for j in range(df['quantidade_pontos'][i] - 1):
        tmp = False
        if abs(float(dados[i]['longitude'][j]) - float(dados[i]['longitude'][j+1])) > 0.000003:
            variacao_lon.append(1)
            tmp = True
            break
    if tmp == False:
        variacao_lon.append(0)
        
df['variacao_lon'] = variacao_lon

In [ ]:
#Coloca variacao da latitude no DataFrame
variacao_lat = []
tmp = False
for i in range(n):
    for j in range(df['quantidade_pontos'][i] - 1):
        tmp = False
        if abs(float(dados[i]['latitude'][j]) - float(dados[i]['latitude'][j+1])) > 0.000003:
            variacao_lat.append(1)
            tmp = True
            break
    if tmp == False:
        variacao_lat.append(0)

df['variacao_lat'] = variacao_lat

In [ ]:
#Coloca a variação da ignicao no DataFrame
variacao_ignicao = []
for i in range(n):
    for j in range(df['quantidade_pontos'][i] - 1):
        tmp = False
        if dados[i]['ignicao'][j] != dados[i]['ignicao'][j+1]:
            variacao_ignicao.append(1)
            tmp = True
            break
    if tmp == False:
        variacao_ignicao.append(0)

df['variacao_ignicao'] = variacao_ignicao

In [ ]:
#Colocando a variacao do Odometro no DataFrame
variacao_odometro = []
tmp = False
for i in range(n):
    for j in range(df['quantidade_pontos'][i] - 1):
        tmp = False
        if type(dados[i]['odometro'][0]) != str:
            if abs(float(dados[i]['odometro'][j]) - float(dados[i]['odometro'][j+1])) > 0.1:
                variacao_odometro.append(1)
                tmp = True
                break
    if tmp == False:
        variacao_odometro.append(0)
df['variacao_odometro'] = variacao_odometro

In [ ]:
#Colocando a variacao de velocidade no DataFrame
variacao_velocidade = []
for i in range(n):
    for j in range(df['quantidade_pontos'][i] - 1):
        tmp = False
        if dados[i]['velocidade'][j] != dados[i]['velocidade'][j+1]:
            variacao_velocidade.append(1)
            tmp = True
            break
    if tmp == False:
        variacao_velocidade.append(0)
df['variacao_velocidade'] = variacao_velocidade

In [ ]:
#Colocando tensoes da bateria no DataFrame
tensao_bateria_min = []
tensao_bateria_max = []
tensao_bateria_min_igon = []
tensao_bateria_max_igon = []
tensao_bateria_min_igoff = []
tensao_bateria_max_igoff = []
variacao_tensao_bateria = []

for i in range(n):
    min, min_igon, min_igoff = 50, 50, 50
    max, max_igon, max_igoff = 0, 0, 0 
    
    for j in range(df['quantidade_pontos'][i]):
        if float(dados[i]['bateria'][j]) < min:
            min = float(dados[i]['bateria'][j])
        if float(dados[i]['bateria'][j]) > max:
            max = float(dados[i]['bateria'][j])
        if float(dados[i]['bateria'][j]) < min_igon and dados[i]['ignicao'][j] == 1:
            min_igon = float(dados[i]['bateria'][j])
        if float(dados[i]['bateria'][j]) > max_igon and dados[i]['ignicao'][j] == 1:
            max_igon = float(dados[i]['bateria'][j])
        if float(dados[i]['bateria'][j]) < min_igoff and dados[i]['ignicao'][j] == 0:
            min_igoff = float(dados[i]['bateria'][j])
        if float(dados[i]['bateria'][j]) > max_igoff and dados[i]['ignicao'][j] == 0:
            max_igoff = float(dados[i]['bateria'][j])
            
    tensao_bateria_min.append(min)
    tensao_bateria_max.append(max)
    if min_igon == 50:
        tensao_bateria_min_igon.append(0)
    else:
        tensao_bateria_min_igon.append(min_igon)
        
    tensao_bateria_max_igon.append(max_igon)
    tensao_bateria_min_igoff.append(min_igoff)
    tensao_bateria_max_igoff.append(max_igoff)
    variacao_tensao_bateria.append(max - min)
    
df['tensao_bateria_min'] = tensao_bateria_min
df['tensao_bateria_max'] = tensao_bateria_max
df['tensao_bateria_min_igon'] = tensao_bateria_min_igon
df['tensao_bateria_max_igon'] = tensao_bateria_max_igon
df['tensao_bateria_min_igoff'] = tensao_bateria_min_igoff
df['tensao_bateria_max_igoff'] = tensao_bateria_max_igoff
df['variacao_tensao_bateria'] = variacao_tensao_bateria

In [ ]:
#Coloca bateriaRMS no DataFrame
bateriaRMS = []
for i in range(n):
    rms = 0
    for j in range(df['quantidade_pontos'][i]):
        rms = rms + float(dados[i]['bateria'][j])**2
    rms = rms / df['quantidade_pontos'][i]
    bateriaRMS.append(rms ** 0.5)

df['bateria_RMS'] = bateriaRMS

In [ ]:
#Colocando pulos de idx no DataFrame
maior_pulo_idx_positivo = []
maior_pulo_idx_negativo = []
numero_pulos_idx = []
    
for i in range(n):
    pos, cont = 0, 0
    neg = 99999
    for j in range(df['quantidade_pontos'][i] - 1):
        if dados[i]['idx_memoria'][j+1] > dados[i]['idx_memoria'][j] + 1:
            cont = cont + 1
            if pos < dados[i]['idx_memoria'][j+1] - dados[i]['idx_memoria'][j]:
                pos = dados[i]['idx_memoria'][j+1] - dados[i]['idx_memoria'][j]
        if dados[i]['idx_memoria'][j+1] < dados[i]['idx_memoria'][j]:
            cont = cont + 1
            if neg > dados[i]['idx_memoria'][j+1] - dados[i]['idx_memoria'][j]:
                neg = dados[i]['idx_memoria'][j+1] - dados[i]['idx_memoria'][j]
    maior_pulo_idx_positivo.append(pos)
    if neg == 99999:
        maior_pulo_idx_negativo.append(0)
    else:
        maior_pulo_idx_negativo.append(neg)
    numero_pulos_idx.append(cont)
    print(i)
df['maior_pulo_negativo_idx'] = maior_pulo_idx_negativo
df['maior_pulo_positivo_idx'] = maior_pulo_idx_positivo
df['numero_pulos_idx'] = numero_pulos_idx

In [ ]:
#Colocando quantidade de pontos do gps no DataFrame
quantidade_pontos_gps_valido = []
quantidade_pontos_gps_invalido = []

for i in range(n):
    q_valido = 0
    for j in range(df['quantidade_pontos'][i]):
        if dados[i]['sinal_gps'][j] == 1:
            q_valido = q_valido + 1
    quantidade_pontos_gps_valido.append(q_valido)
    quantidade_pontos_gps_invalido.append(df['quantidade_pontos'][i] - q_valido)

df['quantidade_pontos_gps_valido'] = quantidade_pontos_gps_valido
df['quantidade_pontos_gps_invalido'] = quantidade_pontos_gps_invalido

In [ ]:
df.to_csv('especilista_sem_label.csv')

# Rotulação dos dados

In [ ]:
#falhas = pd.read_csv('sinapse-deteccao-falhas.falhas.csv')
falhas = pd.read_csv('rel_falhas_tudo.csv')
df = pd.read_csv('especilista_sem_label.csv')
df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
#Formata o campo data nas Falhas
def mes_StringToNumString(x):
    return {
        'Jan': '01',
        'Feb': '02',
        'Mar': '03',
        'Apr': '04',
        'May': '05',
        'Jun': '06',
        'Jul': '07',
        'Ago': '08',
        'Sep': '09',
        'Oct': '10',
        'Nov': '11',
        'Dec': '12'
    }.get(x, '00')  

for i in range(len(falhas)):
    s = falhas['data'][i].split(',')
    tmp = s[0].split(' ')
    mes = tmp[0] 
    mes = mes_StringToNumString(mes)
    dia = tmp[1]
    ano = s[1].split(' ')[1]
    falhas['data'][i] = dia + '/' + mes + '/' + ano
    print(i)
falhas.to_csv('Falhas28-11_transf.csv')

In [ ]:
%%time
#Coloca qual falha existe no DataFrame
fault = []
for i in range(len(df)):
    for j in range(len(falhas)):
        if str(df['serial'][i]) == falhas['serial'][j] and df['data'][i] == falhas['data'][j]:
            index = j
            break
    print(i)
    fault.append(falhas['falhas'][index])
df['falhas'] = fault

In [ ]:
#Coloca possui_falhas no DataFrame
possui_falhas = []
for i in range(len(df)):
    if df['falhas_novo'][i] == '[]':
        possui_falhas.append(0)
    else:
        possui_falhas.append(1)

df['possui_falhas'] = possui_falhas

In [ ]:
df.to_csv('especialista.csv')